In [2]:
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.externals import joblib
from github import Github
import gensim
from langdetect import detect
from collections import Counter
import re

In [9]:
popular_repos = pd.read_csv('popular_repos.csv')
popular_repos = popular_repos.reponame

In [10]:
# github object using my personal access token
g = Github('dc346ff7262b4b55dcc8ea6abe35c9c3170d415b')

readme_files = []
langs = []
# for each popular repo
for repo_name in popular_repos:
    
    # get the repository
    repo = g.get_repo(repo_name)
    
    # get the readme file (Shaun's code)
    try:
        readMe = repo.get_contents('README.md')
        readMe = readMe.decoded_content
        readMe = readMe.decode("utf-8")
        # gets rid of hyperlinks
        readMe =re.sub(r'http\S+', '', readMe)
        readMe =re.sub(r'<.*?>', '', readMe)
        readMe =re.sub(r'([\(\[]).*?([\)\]])', '', readMe)
        readMe = re.sub("[^a-zA-Z]+", " ",readMe)
        readme_files.append(readMe)
    except:
        readMe ="No README.md found."
        readme_files.append(readMe)
    try:
        lang = detect(readMe)
        langs.append(lang)
    except:
        langs.append('NA')

In [11]:
# check length 
len(popular_repos) == len(readme_files) == len(langs)

True

In [12]:
## create a pandas dataframe holding all the variables for tracking the 
# 'popularity' measures of the repos plus the readMe files as strings:
readMesAndPopularity = pd.DataFrame(
    {'username/repoName': popular_repos,
     'readMes': readme_files,
     'language': langs
    })

In [14]:
readMesAndPopularity.head()

,username/repoName,readMes,language
0,SeleniumHQ/selenium,Selenium Selenium is an umbrella project encap...,en
1,joshwcomeau/guppy,Guppy A friendly application manager and task...,en
2,KieSun/InterviewMap,Interview Map This is a map that can help you...,en
3,diegohaz/reakit,Toolkit for building a href a href and a href...,en
4,facebookincubator/xar,XAR a href alt CircleCI Status src a href alt...,en


In [15]:
readMesAndPopularity.to_csv('ReadMes4Song.csv')

In [64]:
readMesAndPopularity

,username/repoName,readMes,language
0,alcor/itty-bitty,itty bitty site itty bitty takes html compres...,en
1,Athari/CssGitHubWindows,No README.md found.,en
2,google/gif-for-cli,Takes in a GIF short video or a query to the ...,en
3,django/django,No README.md found.,en
4,Glench/fuzzyset.js,No README.md found.,en
5,astaxie/build-web-application-with-golang,Multiple Language Versions Donate AliPay Engl...,en
6,nbedos/termtosvg,termtosvg termtosvg is a Linux terminal recor...,en
7,judasn/IntelliJ-IDEA-Tutorial,IntelliJ IDEA Watch Star Fork Fork PDF Ctrl G...,en
8,libfann/fann,Fast Artificial Neural Network Library FANN F...,en
9,ApolloAuto/apollo,We choose to go to the moon in this decade an...,en


In [59]:
re.sub("[^a-zA-Z]+", " ",readMesAndPopularity.readMes[14])

' Java Java Star Java Standard Project Description Java IDE Sonar Java Core Library Project Description Java Tools Project Description Manual JMAP JVM JVM JMX Contributing VJTools viptech Bug Developers '